## Fine Tuning

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os

# --- Load API Key ---
load_dotenv(override=True, dotenv_path="../.env")
my_api_key = os.getenv("OPENAI_API_KEY")


client = OpenAI(api_key=my_api_key)

In [2]:
with open("data/training_data.jsonl", "rb") as f:
    file_obj = client.files.create(file=f, purpose="fine-tune")

print("Uploaded file ID:", file_obj.id)

with open("data/validation_data.jsonl", "rb") as f:
    file_obj = client.files.create(file=f, purpose="fine-tune")

print("Uploaded file ID:", file_obj.id)

Uploaded file ID: file-YVHksMC2k6pWBJe87b4dZa
Uploaded file ID: file-QDFL1U8Ea63tv48YrCN5rJ


In [ ]:
# Create a fine-tuning job
job = client.fine_tuning.jobs.create(
    training_file="file-RcKKt5VwXejg8W9hhXxWKp",       # The file ID you uploaded
    validation_file="file-6jbUexPFeFLpCcAgzAMHo7",     # Optional
    model="gpt-4.1-2025-04-14",               # or "gpt-3.5-turbo", etc.
    suffix="brand-customer-support"       # Optional model name suffix
)

print("Fine-tune job created:", job.id)

In [ ]:
jobs = client.fine_tuning.jobs.list(limit=5)
for j in jobs.data:
    print(j.id, j.status, j.fine_tuned_model)

ftjob-4zpZcbnjrzv13CWzrBDOC8ZX running None


In [28]:
def ask_question_without_finetuning(prompt):
    print(f"User asked: {prompt}")
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
            {"role": "user", "content": prompt}
        ]
    )
    print (response)
    return response.choices[0].message.content  

def ask_question_with_finetuning(prompt):
    print(f"User asked: {prompt}")
    response = client.chat.completions.create(
        model="ft:gpt-4.1-2025-04-14:personal:brand-voice-support:CPjcskrj",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
            {"role": "user", "content": prompt}
        ]
    )
    print (response)
    return response.choices[0].message.content  

In [30]:
import time
while True:
    # Ask user for a question
    user_prompt = input("Ask something: ")

    if (user_prompt.lower() != 'quit'):
        # Get and print the response
        response = ask_question_without_finetuning(user_prompt)
        print("\n[WITHOUT FINE TUNING] OpenAI says:", response)

        response = ask_question_with_finetuning(user_prompt)
        print("\n[WITH FINE TUNING] OpenAI says:", response)

        # add delay of 3 seconds
        time.sleep(3)
    else:
        break    

User asked: I have not recieved my refund.
ChatCompletion(id='chatcmpl-CPjxJa6RyUQ7T9YxtVi1jQ2tOSosA', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I’m sorry you’re dealing with this. I can help you get it sorted. To start, could you share a few details?\n\n- Order number\n- Date you requested the refund and the amount\n- Payment method (credit/debit card, PayPal, store credit, etc.)\n- Any refund reference or transaction ID you’ve received\n- Whether you’ve checked your bank/card statement and spam folder for a refund\n\nIn the meantime, here are quick steps you can take:\n1) Check the vendor’s site (Orders/Refunds section) and your email for a refund confirmation or reference.\n2) Look at your bank or payment app statement to see if the refund is marked as posted or pending.\n3) If you don’t see it after the typical processing window (often 3–7 business days for cards, up to 14 for some banks/e-wallets), contact the seller’s sup